# BERT on IMDB Sentiment Analysis

这个文件在IMDB上进行情感分类，熟悉transformers库的使用，测试以下模型的表现：

* BERT

目前的进度：

* 未完成

问题：
* BERT用来分类时后面的模型怎么接？是直接将[CLS]接入线性层，还是将所有token的表示输入一个RNN？

参考：
* [bentrevett/pytorch-sentiment-analysis/6 - Transformers for Sentiment Analysis.ipynb](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb)
* [torchtext使用--Transformer的IMDB情感分析](https://blog.csdn.net/weixin_43301333/article/details/105893946)

## Requirement

## Import

In [1]:
import torch
from torchtext import datasets
from torchtext import data
import numpy as np
import random
from torch import nn,optim
from sklearn import metrics
import torch.nn.functional as F

use_cuda=torch.cuda.is_available()
device=torch.device("cuda" if use_cuda else "cpu")

SEED = 1234
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

## 可供调整的参数

In [40]:
bs=128
d_hidden=256
d_output=2
n_layers=2
bidirectional=True
dropout=0.25
model_name='1'
max_epochs=1
require_improvement=3

## Data

**首先为了使用BERT，我们需要使用配套的tokennizer**

In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(len(tokenizer.vocab))

30522


该分词器中含30522个token。

我们可以简单看一下这个分词器的效果，可以调用分词器的`tokenizer.tokenize`方法，对输入的句子分词。

可以调用`tokenizer.convert_tokens_to_ids`方法,将句子转换成序号.

In [3]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')
print(tokens)
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

['hello', 'world', 'how', 'are', 'you', '?']
[7592, 2088, 2129, 2024, 2017, 1029]


接下来可以看看词典中的特殊字符

In [4]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
print(init_token, eos_token, pad_token, unk_token)
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100


Transformer类型的模型均有最大文本长度，在词典`tokenizer.max_model_input_sizes`存储着多个模型对应的最大长度。

In [5]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


可以看到最大长度是512。

**我们需要定义一个分词器，以传入field。**

注意这里需要留两位给两个特殊token。

In [6]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

**定义field**，需要注意的是我们传入了分词方法和预处理方法：`tokenize = tokenize_and_cut`, `preprocessing = tokenizer.convert_tokens_to_ids`。

`use_vocab = False`是由于我们已经建立的词典，另外需要传入特殊字符的序号。注意这里传入的是序号而非字符串。

In [7]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.long)

**载入数据并进行划分。**

In [8]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(split_ratio=0.8)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')
#一个样本
print(vars(train_data.examples[0]))
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[0])['text'])
print(tokens)

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000
{'text': [2023, 3185, 2038, 2288, 2000, 2022, 2028, 1997, 1996, 5409, 1045, 2031, 2412, 2464, 2191, 2009, 2000, 4966, 999, 999, 999, 1996, 2466, 2240, 2453, 2031, 13886, 2065, 1996, 2143, 2018, 2062, 4804, 1998, 4898, 2008, 2052, 2031, 3013, 1996, 14652, 1998, 5305, 2135, 5019, 2008, 1045, 3811, 14046, 3008, 2006, 1012, 1012, 1012, 1012, 2021, 1996, 2466, 2240, 2003, 2066, 1037, 6065, 8854, 1012, 2065, 2045, 2001, 2107, 1037, 2518, 2004, 1037, 3298, 27046, 3185, 9338, 1011, 2023, 2028, 2052, 2031, 22057, 2013, 2008, 1012, 2009, 6966, 2033, 1037, 2843, 1997, 1996, 4248, 2666, 3152, 2008, 2020, 2404, 2041, 1999, 1996, 3624, 1005, 1055, 1010, 3532, 5896, 3015, 1998, 7467, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2069, 21082, 3494, 1999, 1996, 2878, 3185, 2001, 1996, 15812, 1998, 13570, 1012, 1996, 2717, 1997, 1996, 2143, 1010, 2071, 2031, 4089, 2042, 2081, 2011, 2690

尽管不需要建立词典，但是标签的序号需要确定。

In [9]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


**Iterator需要建立。**

In [33]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = bs, 
    device = device)

## Model

In [13]:
from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

对于BertModel来说，它的返回由四个部分组成：
* last_hidden_state ：（batch,seq,hidden_size）
整个输入的句子每一个token的隐层输出，也是我们这里将要用到的，可以将它作为embedding的替代
* pooler_output：（batch,hidden_size）
输入句子第一个token的最高隐层。也就是[CLS]标记提取到的最终的句对级别的抽象信息。对于BERT的预训练来说，这个隐层信息将作为Next Sentence prediction任务的输入。然而我们这里将不会用到它，因为它对于情感分析来说效果不是很好。
This output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence.
* hidden_states：
一个元组，里面每个元素都是(batch,seq,hidden_size) 大小的FloatTensor，分别代表每一层的隐层和初始embedding的和
* attentions:
一个元组，里面每个元素都是(batch,num_heads,seq,seq) 大小的FloatTensor，分别表示每一层的自注意力分数。

在本实验中，我们仅用BERT作为一个特征抽取器，固定参数，不去进行微调。

In [35]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        #固定参数，不对BERT进行微调
        for p in self.parameters():
            p.requires_grad = False
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            #固定参数，不对BERT进行微调
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, hidden_dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output
    
model = BERTGRUSentiment(bert,
                         d_hidden,
                         d_output,
                         n_layers,
                         bidirectional,
                         dropout)

In [36]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


测试一下能否跑通

In [37]:
with torch.no_grad():
    criterion = nn.CrossEntropyLoss()
    if use_cuda:
        model.to(device)
        criterion.to(device)
    for batch in train_iterator:
        #print(batch.text.shape)
        #print(batch.label)
        logits=model(batch.text)
        print(logits.shape)
        criterion(logits,batch.label)
        break

torch.Size([4, 2])


## Training

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
def train(model, train_iter, dev_iter, test_iter):
    model.to(device)
    model.train()
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    #criterion = nn.BCEWithLogitsLoss()
    if use_cuda:
        criterion.cuda()

    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    #writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d.%H.%M', time.localtime())+'_'+which_data+'_'+which_model+'_'+which_task+'_'+exp_number)
    
    for epoch in range(max_epochs):
        start_time = time.time()
        train_loss=0
        train_correct=0
        # scheduler.step() # 学习率衰减
        for i, batch in enumerate(train_iter):
            optimizer.zero_grad()
            x,l=batch.text
            y=batch.label
            outputs = model(x,l)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            #训练集的准确率
            preds = torch.max(outputs, 1)[1]
            #preds = torch.round(torch.sigmoid(outputs))
            train_correct+=(y==preds).sum()
            train_loss+=loss.item()
        train_loss/=len(train_iterator)   #train_loss
        train_acc=train_correct/len(train_iterator.dataset)   #train_acc
            
        #验证集
        dev_acc, dev_loss = evaluate(model, dev_iter)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        if dev_loss < dev_best_loss:
            dev_best_loss = dev_loss
            improve = '*'
            last_improve=epoch
            torch.save(model.state_dict(),model_name+'.pth')
        else:
            improve = ''
        msg = 'Epoch: {0:>3},  Epoch Time: {1}m {2}s,  Train Loss: {3:>5.2},  Train Acc: {4:>6.2%},  Val Loss: {5:>5.2},  Val Acc: {6:>6.2%} {7}'
        print(msg.format(epoch+1,epoch_mins, epoch_secs,train_loss, train_acc, dev_loss, dev_acc, improve))
        #writer.add_scalar("loss/train", loss.item(), total_batch)
        #writer.add_scalar("loss/dev", dev_loss, total_batch)
        #writer.add_scalar("acc/train", train_acc, total_batch)
        #writer.add_scalar("acc/dev", dev_acc, total_batch)

        if epoch - last_improve > require_improvement:
            # 验证集loss超过1epoch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break
    #writer.close()
    #训练跑完了，使用最佳模型测试
    model.load_state_dict(torch.load(model_name+'.pth'))
    test(model, test_iter)

def evaluate(model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for batch in data_iter:
            x,l=batch.text
            labels=batch.label
            outputs = model(x,l)
            loss = F.cross_entropy(outputs, labels)
            #loss=criterion(outputs,labels)
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = torch.max(outputs, 1)[1].cpu().numpy()
            #predic=torch.round(torch.sigmoid(outputs)).cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)
    model.train()
    acc = metrics.accuracy_score(labels_all, predict_all)
    
    if test:
        report = metrics.classification_report(labels_all, predict_all, labels=[0,1],target_names=['pos','neg'], digits=4,output_dict=True)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    
    return acc, loss_total / len(data_iter)


def test(model, test_iter):
    test_acc, test_loss, test_report, test_confusion = evaluate(model, test_iter, test=True)
    msg = 'Test Loss: {0:>5.2},  Test Acc: {1:>6.2%}'
    print(msg.format(test_loss, test_acc))
    print("Precision, Recall and F1-Score...")
    print(test_report)
    print("Confusion Matrix...")
    print(test_confusion)

In [ ]:
#original
train(model,train_iterator,valid_iterator,test_iterator)